In [1]:
import pandas as pd
import os
from eunjeon import Mecab
from ast import literal_eval

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#8_clean_text'}.xlsx")
df = pd.read_excel(xname, index_col=0)
mecab = Mecab()
corpus = ''
for i in range(len(df)):
    text = literal_eval(df.loc[i, 'text'])
    corpus = corpus + ( "\n".join([" ".join([f for f in mecab.nouns(e) if len(f)>1]) for e in text]))

with open('corpus.txt', 'w',encoding='utf-8') as f:
    for c in corpus:
        f.write("%s" % c)

In [2]:
import logging
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from gensim.models import word2vec
from pathlib import Path
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


data = word2vec.LineSentence('corpus.txt')
model = word2vec.Word2Vec(size=200, window=2, hs=1, min_count=2, sg=1)
model.build_vocab(data)

for epoch in range(1):

    model.train(data, total_examples=model.corpus_count, epochs=model.iter)

    model.alpha -= 0.002 # decrease the learning rate

    model.min_alpha = model.alpha # fix the learning rate, no decay

model.wv.save_word2vec_format('w2v_0829.txt', binary=False),
model.save("w2v_0829.model")
print("ok")

2020-08-29 16:28:49,851 : INFO : collecting all words and their counts
2020-08-29 16:28:49,857 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-29 16:28:49,894 : INFO : PROGRESS: at sentence #10000, processed 119543 words, keeping 8989 word types
2020-08-29 16:28:49,936 : INFO : PROGRESS: at sentence #20000, processed 244375 words, keeping 12414 word types
2020-08-29 16:28:49,974 : INFO : PROGRESS: at sentence #30000, processed 367940 words, keeping 15186 word types
2020-08-29 16:28:50,012 : INFO : PROGRESS: at sentence #40000, processed 481953 words, keeping 17325 word types
2020-08-29 16:28:50,050 : INFO : PROGRESS: at sentence #50000, processed 600054 words, keeping 19501 word types
2020-08-29 16:28:50,091 : INFO : PROGRESS: at sentence #60000, processed 719094 words, keeping 21355 word types
2020-08-29 16:28:50,129 : INFO : PROGRESS: at sentence #70000, processed 837757 words, keeping 23049 word types
2020-08-29 16:28:50,168 : INFO : PROGRESS: at s

2020-08-29 16:29:25,081 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-29 16:29:25,110 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-29 16:29:25,122 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-29 16:29:25,123 : INFO : EPOCH - 5 : training on 2314205 raw words (2281400 effective words) took 6.2s, 367190 effective words/s
2020-08-29 16:29:25,124 : INFO : training on a 11571025 raw words (11407981 effective words) took 30.3s, 376444 effective words/s
2020-08-29 16:29:25,124 : INFO : storing 25207x200 projection weights into w2v_0829.txt
2020-08-29 16:29:28,147 : INFO : saving Word2Vec object under w2v_0829.model, separately None
2020-08-29 16:29:28,148 : INFO : not storing attribute vectors_norm
2020-08-29 16:29:28,148 : INFO : not storing attribute cum_table
2020-08-29 16:29:28,778 : INFO : saved w2v_0829.model


ok


In [5]:
model = word2vec.Word2Vec.load("w2v_0829.model")
d = model.wv.most_similar(positive=["병원"])
for (x, y) in d:
    print("%s - %f" % (x,y))

2020-08-29 16:30:02,981 : INFO : loading Word2Vec object from w2v_0829.model
2020-08-29 16:30:03,467 : INFO : loading wv recursively from w2v_0829.model.wv.* with mmap=None
2020-08-29 16:30:03,467 : INFO : setting ignored attribute vectors_norm to None
2020-08-29 16:30:03,468 : INFO : loading vocabulary recursively from w2v_0829.model.vocabulary.* with mmap=None
2020-08-29 16:30:03,469 : INFO : loading trainables recursively from w2v_0829.model.trainables.* with mmap=None
2020-08-29 16:30:03,469 : INFO : setting ignored attribute cum_table to None
2020-08-29 16:30:03,470 : INFO : loaded w2v_0829.model
2020-08-29 16:30:03,530 : INFO : precomputing L2-norms of word weight vectors


응급실 - 0.718561
신경외과 - 0.676160
진료실 - 0.648930
내원 - 0.647565
일반외과 - 0.641778
외과 - 0.641260
복음 - 0.632771
강남병원 - 0.630099
안암 - 0.628555
초진 - 0.627105


In [4]:
#0829 space-correct